In [1]:
from dotenv import load_dotenv
import os
load_dotenv()
groq_api_key = os.getenv("GROQ_API_KEY")

In [81]:
from langchain_groq import ChatGroq

llm= ChatGroq(
    model="llama3-8b-8192",
    max_tokens= 2000,
    temperature=0,
    timeout=None,
    max_retries=2,
    stream=False,
    response_format={"type": "json_object"},
    stop=None,
)
    


c:\Users\utente\OneDrive\Bureau\project\project\Lib\site-packages\langchain_groq\chat_models.py:362: UserWarning: WARNING! stream is not default parameter.
                    stream was transferred to model_kwargs.
                    Please confirm that stream is what you intended.
  warnings.warn(
c:\Users\utente\OneDrive\Bureau\project\project\Lib\site-packages\langchain_groq\chat_models.py:362: UserWarning: WARNING! response_format is not default parameter.
                    response_format was transferred to model_kwargs.
                    Please confirm that response_format is what you intended.
  warnings.warn(


we need to format the input json into the same structure so it will be parsed correctly
it should be formatted like json2 or 3 not 1 

In [102]:
import json
from pathlib import Path
from rich.console import Console
from rich.markdown import Markdown

console = Console()

pretty_print_json = lambda x: console.print(Markdown(f"```json\n{json.dumps(json.loads(x), indent=2)}\n```"))

transcripts = Path(r"C:\Users\utente\OneDrive\Bureau\project\data")

transcript_file = "newsample1.json"
transcript_path = transcripts / transcript_file

with transcript_path.open("r") as file:
    transcript = file.read()

transcript_dict = json.loads(transcript)

call_date = transcript_dict['call_date']
call_time = transcript_dict['call_time']

pretty_print_json(transcript)


                                                                                                                   
 {                                                                                                                 
   "call_transcript": [                                                                                            
     "CSR: Welcome to ABC Bank. This is Sarah speaking. How can I help you today?",                                
     "Customer: Hi, I noticed some unauthorized transactions on my credit card. Can you help me figure this out?", 
     "CSR: I'm sorry to hear that. Could you please provide your full name and the last four digits of your card s 
 I can pull up your account?",                                                                                     
     "Customer: Sure, it's Emily Carter, and the last four digits are 4567.",                                      
     "CSR: Thank you, Ms. Carter. I see three transactions from yesterday totaling $1,200 at stores in New York.   
 Were these authorized by you?",                                                                                   
     "Customer: No, I wasn't even in New York yesterday!",                                                         
     "CSR: Understood. I'll mark those transactions as disputed and issue a temporary credit while we investigate. 
 I'll also block your card and issue a replacement. Would that work for you?",                                     
     "Customer: Yes, please. How long will the investigation take?",                                               
     "CSR: It typically takes 7-10 business days. I'll send you an email with a claim reference number and next    
 steps. Is there anything else I can assist you with?",                                                            
     "Customer: No, thank you for your help.",                                                                     
     "CSR: You're welcome, Ms. Carter. Thank you for banking with ABC Bank, and have a great day!"                 
   ],                                                                                                              
   "call_ID": "23456",                                                                                             
   "CSR_ID": "SCarter456",                                                                                         
   "call_date": "2024-02-06",                                                                                      
   "call_time": "14:23:45"                                                                                         
 }                                                                                                                 
                                                                                                                   

In [103]:
import re
import json
from pydantic import BaseModel, Field , ValidationError
from typing import List
from langchain.prompts import ChatPromptTemplate
from langchain.output_parsers import PydanticOutputParser
from rich.console import Console
from rich.markdown import Markdown
from pathlib import Path
from langchain_ollama import ChatOllama
from langchain.chains import LLMChain
from langchain_core.output_parsers import JsonOutputParser


In [104]:
from langchain.prompts import ChatPromptTemplate

# Define the output schema
output_schema = {
    "properties": {
        "call_summary": {
            "description": "Call transcript summary: ",
            "title": "Call Summary",
            "type": "string"
        },
        "key_takeaways": {
            "description": "Call transcript key takeaways: ",
            "items": {"type": "string"},
            "title": "Key Takeaways",
            "type": "array"
        },
        "follow_up_actions": {
            "description": "Call transcript key action items: ",
            "items": {"type": "string"},
            "title": "Follow Up Actions",
            "type": "array"
        }
    },
    "required": ["call_summary", "key_takeaways", "follow_up_actions"]
}

# Define the template
summarization_template = """

Please provide a summary of the following call transcript provided between <transcript></transcript> tags. 
Capture key takeaways and specific follow-up actions. 
Skip the preamble and go straight to the answer.

<transcript>{transcript}</transcript>


Format your response using the following JSON schema:
{output_schema}

Place your response between <output></output> tags.
"""
parser = JsonOutputParser()


# Create the prompt template
summarization_prompt = ChatPromptTemplate.from_template(
    summarization_template,
    partial_variables={
        "output_schema": output_schema,
        },
)



In [105]:
chain= ( summarization_prompt
    | llm
    | parser
)

response= chain.invoke(transcript)

In [106]:
print(response)

{'output': {'properties': {'call_summary': {'description': 'Call transcript summary: ', 'title': 'Call Summary', 'type': 'string'}, 'key_takeaways': {'description': 'Call transcript key takeaways: ', 'items': {'type': 'string'}, 'title': 'Key Takeaways', 'type': 'array'}, 'follow_up_actions': {'description': 'Call transcript key action items: ', 'items': {'type': 'string'}, 'title': 'Follow Up Actions', 'type': 'array'}}, 'required': ['call_summary', 'key_takeaways', 'follow_up_actions'], 'call_summary': 'Customer reported unauthorized transactions on credit card, totaling $1,200. CSR marked transactions as disputed, issued temporary credit, and blocked card.', 'key_takeaways': ['Unauthorized transactions on credit card', 'Totaling $1,200', 'Customer was not in New York at the time of transactions', 'CSR marked transactions as disputed and issued temporary credit', 'CSR blocked card and issued replacement'], 'follow_up_actions': ['Investigate transactions and resolve dispute', 'Send em

In [ ]:
desired_response = {
    'call_summary': response['output'].get('call_summary'),
    'key_takeaways': response['output'].get('key_takeaways'),
    'follow_up_actions': response['output'].get('follow_up_actions'),
}

# Print the result
print("Extracted Response:")
print(desired_response)

Extracted Response:
Customer reported unauthorized transactions on credit card, totaling $1,200. CSR marked transactions as disputed, issued temporary credit, and blocked card.


In [139]:
summary=desired_response['call_summary']
print(summary)
type(summary)

Customer reported unauthorized transactions on credit card, totaling $1,200. CSR marked transactions as disputed, issued temporary credit, and blocked card.


str

In [149]:
import time 

def stream_characters(text,delay):
    for char in text:
        yield char
        time.sleep(delay)

# Example usage
for char in stream_characters(summary,0.02):
    print(char, end="", flush=True)  # Print each 

Customer reported unauthorized transactions on credit card, totaling $1,200. CSR marked transactions as disputed, issued temporary credit, and blocked card.

In [114]:
type(desired_response)

dict

In [116]:
import json
import os

desired_response["call_ID"] = transcript_dict['call_ID']
desired_response["CSR_ID"] = transcript_dict['CSR_ID']
desired_response["call_date"] = call_date
desired_response["call_time"] = call_time

response = json.dumps(desired_response, indent=2)
pretty_print_json(response)


                                                                                                                   
 {                                                                                                                 
   "call_summary": "Customer reported unauthorized transactions on credit card, totaling $1,200. CSR marked        
 transactions as disputed, issued temporary credit, and blocked card.",                                            
   "key_takeaways": [                                                                                              
     "Unauthorized transactions on credit card",                                                                   
     "Totaling $1,200",                                                                                            
     "Customer was not in New York at the time of transactions",                                                   
     "CSR marked transactions as disputed and issued temporary credit",                                            
     "CSR blocked card and issued replacement"                                                                     
   ],                                                                                                              
   "follow_up_actions": [                                                                                          
     "Investigate transactions and resolve dispute",                                                               
     "Send email with claim reference number and next steps to customer",                                          
     "Monitor customer's account for any further issues"                                                           
   ],                                                                                                              
   "call_ID": "23456",                                                                                             
   "CSR_ID": "SCarter456",                                                                                         
   "call_date": "2024-02-06",                                                                                      
   "call_time": "14:23:45"                                                                                         
 }                                                                                                                 
                                                                                                                   

In [117]:
sanitized_call_date = call_date.replace(':', '-').replace(' ', '_')
sanitized_call_time = call_time.replace(':', '-').replace(' ', '_')

In [118]:
output_directory = "output"
if not os.path.exists(output_directory):
    os.makedirs(output_directory)

file_path = os.path.join(output_directory, f"Call Summary {sanitized_call_date} {sanitized_call_time}.json")

In [119]:
with open(file_path, 'w') as file:
    file.write(response)

print(f"Transcript summary written to local file: {file_path}")

Transcript summary written to local file: output\Call Summary 2024-02-06 14-23-45.json


In [125]:
from functools import partial
from langchain_core.runnables import RunnableLambda

one_shot_example = """

Evaluation:
{{
  "Communication_Skills": {{
    "score": "Medium",
    "score_explanation": "The CSR communicated adequately but could have been more concise and used more empathetic language when addressing the customer's frustration."
  }},
  "Problem_Resolution": {{
    "score": "High",
    "score_explanation": "The CSR resolved the customer's issue by offering an immediate refund and finding an alternative flight that met the customer's needs."
  }},
  "Product_Knowledge": {{
    "score": "High",
    "score_explanation": "The CSR showed a strong understanding of the company's policies and was able to clearly explain the details of the flight refund process."
  }},
  "Professionalism": {{
    "score": "High",
    "score_explanation": "The CSR maintained a professional demeanor throughout the conversation, addressing the customer's concerns respectfully and patiently."
  }},
  "Problem_Escalation": {{
    "score": "Medium",
    "score_explanation": "The CSR acknowledged the issue but did not transfer the call to a supervisor until further prompting from the customer."
  }},
  "Resolution_Follow_Up": {{
    "score": "High",
    "score_explanation": "The CSR mentioned the follow-up procedure and confirmed that an email would be sent to the customer for further details."
  }},
  "Efficiency": {{
    "score": "Medium",
    "score_explanation": "The call was handled in a reasonable timeframe but included pauses to verify flight details, causing some delays."
  }},
  "Adherence_to_Policies_and_Procedures": {{
    "score": "High",
    "score_explanation": "The CSR accurately followed all company policies, ensuring that the refund and rebooking were processed correctly."
  }},
  "Technical_Competence": {{
    "score": "Medium",
    "score_explanation": "The CSR navigated the systems well but took longer than expected to find the necessary information for the refund."
  }},
  "Customer_Satisfaction": {{
    "score": "High",
    "score_explanation": "The customer expressed appreciation for the quick response and the options provided, despite initial frustration."
  }},
  "Language_Proficiency": {{
    "score": "High",
    "score_explanation": "The CSR used clear and professional language throughout the call, ensuring that the customer understood the process."
  }},
  "Conflict_Resolution": {{
    "score": "Medium",
    "score_explanation": "The CSR managed the customer's irritation well, though some de-escalation techniques could have been more proactive."
  }}
}}

"""

assessment_template = f"""
{{call_transcript}}

{one_shot_example}
Now evaluate the given call transcript and provide the output in the same format as the example.
Ensure that the evaluation uses the exact category names and underscores as shown in the example.
Ensure that the output is only JSON without any additional text.
"""

assessment_prompt = ChatPromptTemplate.from_template(assessment_template)

def process_transcript(transcript: str) -> str:
    """
    Extracts and formats the call transcript from the input JSON.
    """
    json_transcript = json.loads(transcript)
    call_transcript = "\n".join(json_transcript.get("call_transcript", []))
    return f"Call Transcript:\n{call_transcript}"

def extract_content(message):
    """
    Extracts the content from the message, converting it to a string if necessary.
    """
    if hasattr(message, 'content') and isinstance(message.content, str):
        return message.content
    return str(message)  

assessment_chain = (
    {"call_transcript": RunnableLambda(process_transcript)}
    | assessment_prompt
    | llm
    | RunnableLambda(extract_content)
    |parser
    
)


In [121]:
call=process_transcript(transcript)
print(call)


Call Transcript:
CSR: Welcome to ABC Bank. This is Sarah speaking. How can I help you today?
Customer: Hi, I noticed some unauthorized transactions on my credit card. Can you help me figure this out?
CSR: I'm sorry to hear that. Could you please provide your full name and the last four digits of your card so I can pull up your account?
Customer: Sure, it's Emily Carter, and the last four digits are 4567.
CSR: Thank you, Ms. Carter. I see three transactions from yesterday totaling $1,200 at stores in New York. Were these authorized by you?
Customer: No, I wasn't even in New York yesterday!
CSR: Understood. I'll mark those transactions as disputed and issue a temporary credit while we investigate. I'll also block your card and issue a replacement. Would that work for you?
Customer: Yes, please. How long will the investigation take?
CSR: It typically takes 7-10 business days. I'll send you an email with a claim reference number and next steps. Is there anything else I can assist you with?

In [126]:
call_assessment = assessment_chain.invoke(transcript)


In [154]:
first_key = list(call_assessment.keys())[1]
print(first_key)

Problem_Resolution


In [134]:
call_assessment["call_ID"] = transcript_dict['call_ID']
call_assessment["CSR_ID"] = transcript_dict['CSR_ID']
call_assessment["call_date"] = call_date
call_assessment["call_time"] = call_time
call_assessment_response= json.dumps(call_assessment, indent=2)
pretty_print_json(call_assessment_response)


                                                                                                                   
 {                                                                                                                 
   "Communication_Skills": {                                                                                       
     "score": "High",                                                                                              
     "score_explanation": "The CSR communicated clearly and empathetically, addressing the customer's concerns and 
 providing explanations in a concise manner."                                                                      
   },                                                                                                              
   "Problem_Resolution": {                                                                                         
     "score": "High",                                                                                              
     "score_explanation": "The CSR resolved the customer's issue by marking the transactions as disputed, issuing  
 temporary credit, and blocking the card, providing a clear plan for next steps."                                  
   },                                                                                                              
   "Product_Knowledge": {                                                                                          
     "score": "High",                                                                                              
     "score_explanation": "The CSR demonstrated a strong understanding of the bank's policies and procedures,      
 explaining the process for disputed transactions and card replacement."                                           
   },                                                                                                              
   "Professionalism": {                                                                                            
     "score": "High",                                                                                              
     "score_explanation": "The CSR maintained a professional demeanor throughout the conversation, addressing the  
 customer's concerns respectfully and patiently."                                                                  
   },                                                                                                              
   "Problem_Escalation": {                                                                                         
     "score": "High",                                                                                              
     "score_explanation": "The CSR acknowledged the issue and took immediate action to resolve it, without requiri 
 further escalation."                                                                                              
   },                                                                                                              
   "Resolution_Follow_Up": {                                                                                       
     "score": "High",                                                                                              
     "score_explanation": "The CSR provided clear next steps and confirmed that an email would be sent to the      
 customer with further details."                                                                                   
   },                                                                                                              
   "Efficiency": {                                                                                                 
     "score": "High",                                                                                              
     "score_explanation": "The CSR handled the call effi

In [135]:
sanitized_call_date = call_date.replace(':', '-').replace(' ', '_')
sanitized_call_time = call_time.replace(':', '-').replace(' ', '_')

In [136]:
output_directory = "output" 
output_path = Path(output_directory) / f"Call_score_{sanitized_call_date}_{sanitized_call_time}.json"


In [137]:
try:
    with open(output_path, 'w', encoding='utf-8') as file:
        file.write(call_assessment_response)
    print(f"Transcript score assessment written to local file: {output_path}")
except Exception as e:
    print(f"An error occurred while writing to the file: {e}")

Transcript score assessment written to local file: output\Call_score_2024-02-06_14-23-45.json
